## Appendix 1

In [ ]:
import re  
import pandas as pd  
from time import time  
from collections import defaultdict 
import numpy as np 
import nltk  

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
for chunk in pd.read_csv('/content/drive/My Drive/Data/all-the-news-2-1.csv', chunksize=100):
    df = pd.DataFrame(chunk)
    if df.empty == False:
        break
df.head(5)

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [ ]:
df['article'].isnull().values.sum()

0

In [ ]:
df = df.drop(columns=['date', 'year', 'month', 'day', 'author', 'title', 'url', 'section', 'publication'])

In [ ]:
df.shape

(10, 1)

In [ ]:
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import sent_tokenize


In [ ]:
df['sent_tokenize'] = df['article'].apply(sent_tokenize) 


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-03-12 19:36:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-12 19:36:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-12 19:36:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)


400000

In [ ]:
def clean_text(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = sentence.replace("[^a-zA-Z]", " ")
        cleaned_sentences.append(cleaned_sentence.lower())
    return cleaned_sentences

df['cleaned_sentences'] = df['sent_tokenize'].apply(clean_text)

In [ ]:
df.head()

,date,year,month,day,author,title,article,url,section,publication,sent_tokenize,cleaned_sentences
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox,"[This post is part of Polyarchy, an independen...","[this post is part of polyarchy, an independen..."
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider,[ The Indianapolis Colts made Andrew Luck the ...,[ the indianapolis colts made andrew luck the ...
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters,"[DAVOS, Switzerland (Reuters) - U.S. President...","[davos, switzerland (reuters) - u.s. president..."
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters,[PARIS (Reuters) - Former French president Nic...,[paris (reuters) - former french president nic...
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ,[Paris Hilton arrived at LAX Wednesday dressed...,[paris hilton arrived at lax wednesday dressed...


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

df['cleaned_sentences'] = df['cleaned_sentences'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def get_sentence_vectors(cleaned_sentences):
    sentence_vectors = []
    for sentence in cleaned_sentences:
        v = [word_embeddings.get(w, np.zeros((100,))) for w in sentence.split()]
        if len(v) < 100:
            v += [np.zeros((100,))] * (100 - len(v))
        elif len(v) > 100:
            v = v[:100]
        v = sum(v) / (len(v) + 0.001)
        sentence_vectors.append(v)
    if len(sentence_vectors) == 0:
        sentence_vectors = [np.zeros((100,))]
    return np.array(sentence_vectors, dtype=float)


In [ ]:
df['sentence_vectors'] = df['cleaned_sentences'].apply(get_sentence_vectors)


In [ ]:
df


,date,year,month,day,author,title,article,url,section,publication,sent_tokenize,cleaned_sentences,sentence_vectors
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox,"[This post is part of Polyarchy, an independen...","this post is part of polyarchy, an independent...","[[0.0013481865115068483, 0.0040223598575591085..."
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider,[ The Indianapolis Colts made Andrew Luck the ...,the indianapolis colts made andrew luck the h...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters,"[DAVOS, Switzerland (Reuters) - U.S. President...","davos, switzerland (reuters) - u.s. president ...","[[-0.009109009013288162, 0.005045849290309282,..."
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters,[PARIS (Reuters) - Former French president Nic...,paris (reuters) - former french president nico...,"[[-0.006516935029013967, 0.008048319799661805,..."
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ,[Paris Hilton arrived at LAX Wednesday dressed...,paris hilton arrived at lax wednesday dressed ...,"[[-0.006516935029013967, 0.008048319799661805,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-06-26 00:00:00,2019,6.0,26,NaN,Switzerland's SoftwareONE mandates banks for I...,"ZURICH, June 26 (Reuters) - Swiss software gro...",https://www.reuters.com/article/softwareone-ip...,Funds News,Reuters,"[ZURICH, June 26 (Reuters) - Swiss software gr...","zurich, june 26 (reuters) - swiss software gro...","[[-4.9026509357489815e-05, 0.00212817865317287..."
96,2017-09-26 09:00:02,2017,9.0,26,Rani Molla,Where the software industry is growing outside...,Software companies are contributing an outsize...,https://www.vox.com/2017/9/26/16346960/softwar...,NaN,Vox,[Software companies are contributing an outsiz...,software companies are contributing an outsize...,"[[0.0013738862872201202, 0.007789021922288406,..."
97,2019-05-27 00:00:00,2019,5.0,27,NaN,Will Smith Secretly Watches 'Aladdin' at Calab...,Will Smith is one of the biggest movie stars i...,https://www.tmz.com/2019/05/27/will-smith-alad...,NaN,TMZ,[Will Smith is one of the biggest movie stars ...,will smith is one of the biggest movie stars i...,"[[-0.008990910147165312, 0.006319937091499751,..."
98,2018-04-20 00:00:00,2018,4.0,20,NaN,TMZ Live Cardi B & Bernie Sanders Join Forces ...,HERE'S THE RUNDOWNTristan Thompson Leaves Khlo...,https://www.tmz.com/2018/04/20/tmz-live/,NaN,TMZ,[HERE'S THE RUNDOWNTristan Thompson Leaves Khl...,here's the rundowntristan thompson leaves khlo...,"[[-0.002031379740895813, 0.005046649772683791,..."


**cosine similarities of the sentences**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def compute_similarity(sentence_vectors):
    n = len(sentence_vectors)
    sim_mat = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            if i != j:
                sim_mat[i, j] = cosine_similarity(sentence_vectors[i].reshape(1, -1), sentence_vectors[j].reshape(1, -1))[0, 0]
    return sim_mat

In [ ]:
df['similarity_matrix'] = df['sentence_vectors'].apply(lambda x: compute_similarity(x)).progress_apply(tqdm)

 19%|█▉        | 80/420 [00:09<00:52,  6.42it/s]

In [ ]:
df

,article,sent_tokenize,cleaned_sentences,sentence_vectors,similarity_matrix
0,"This post is part of Polyarchy, an independent...","[This post is part of Polyarchy, an independen...","[this post is part of polyarchy, an independen...","[[-0.06749450702522804, 0.03845283470659632, 0...","[[0.0, 0.8209741526442826, 0.8661343882604524,..."
1,The Indianapolis Colts made Andrew Luck the h...,[ The Indianapolis Colts made Andrew Luck the ...,[ the indianapolis colts made andrew luck the ...,"[[0.03968366248525548, 0.009602262914634343, 0...","[[0.0, 0.868854228025953, 0.9343260113815506, ..."
2,"DAVOS, Switzerland (Reuters) - U.S. President ...","[DAVOS, Switzerland (Reuters) - U.S. President...","[davos, switzerland (reuters) - u.s. president...","[[-0.023857130795725285, -0.057201496999340844...","[[0.0, 0.9510497052142061, 0.9289680290431044,..."
3,PARIS (Reuters) - Former French president Nico...,[PARIS (Reuters) - Former French president Nic...,[paris (reuters) - former french president nic...,"[[-0.026092488294504824, 0.01578283199400815, ...","[[0.0, 0.9452452071803821, 0.9400790618462382,..."
4,Paris Hilton arrived at LAX Wednesday dressed ...,[Paris Hilton arrived at LAX Wednesday dressed...,[paris hilton arrived at lax wednesday dressed...,"[[0.03635012516146811, -0.019238118671135677, ...","[[0.0, 0.9470859283118694, 0.8958751258387555,..."
5,"BERLIN, June 17 (Reuters) - ECB board member B...","[BERLIN, June 17 (Reuters) - ECB board member ...","[berlin, june 17 (reuters) - ecb board member ...","[[-0.017836371216385035, -0.048343966650413996...","[[0.0, 0.9645579331415346, 0.9602312190392982]..."
6,CARACAS (Reuters) - Venezuelan authorities hav...,[CARACAS (Reuters) - Venezuelan authorities ha...,[caracas (reuters) - venezuelan authorities ha...,"[[0.008267677637270963, -0.0019148314182220057...","[[0.0, 0.9180468711823176, 0.8926102809426495,..."
7,If only every day could be like this. You can’...,"[If only every day could be like this., You ca...","[if only every day could be like this., you ca...","[[-0.0172803283239598, 0.042179181784842305, 0...","[[0.0, 0.9406811720981931, 0.9331990333560202,..."
8,"Google I/O, the company's big developer confer...","[Google I/O, the company's big developer confe...","[google i/o, the company's big developer confe...","[[0.0012241876734280456, -0.004594854212485783...","[[0.0, 0.8383653692341118, 0.8196385596407044,..."
9,China is dismissing unfavorable media reports ...,[China is dismissing unfavorable media reports...,[china is dismissing unfavorable media reports...,"[[-0.08900843953635339, -0.06765881264492032, ...","[[0.0, 0.9607754105320491, 0.9722333117894462,..."


In [ ]:
import networkx as nx

def summarize(row):
    sim_mat = row['similarity_matrix']
    sentences = row['cleaned_sentences']
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # Extract top 3 sentences as the summary
    summary = [ranked_sentences[i][1] for i in range(3) if len(sentences) >= 5][:5]
    return ' '.join(summary)

df['summary'] = df.apply(summarize, axis=1)

In [ ]:
df

,article,sent_tokenize,cleaned_sentences,sentence_vectors,similarity_matrix,summary
0,"This post is part of Polyarchy, an independent...","[This post is part of Polyarchy, an independen...","[this post is part of polyarchy, an independen...","[[-0.06749450702522804, 0.03845283470659632, 0...","[[0.0, 0.8209741526442826, 0.8661343882604524,...","in my heart attack parable, if the doctor had ..."
1,The Indianapolis Colts made Andrew Luck the h...,[ The Indianapolis Colts made Andrew Luck the ...,[ the indianapolis colts made andrew luck the ...,"[[0.03968366248525548, 0.009602262914634343, 0...","[[0.0, 0.868854228025953, 0.9343260113815506, ...","much of this comes from drafting successfully,..."
2,"DAVOS, Switzerland (Reuters) - U.S. President ...","[DAVOS, Switzerland (Reuters) - U.S. President...","[davos, switzerland (reuters) - u.s. president...","[[-0.023857130795725285, -0.057201496999340844...","[[0.0, 0.9510497052142061, 0.9289680290431044,...",
3,PARIS (Reuters) - Former French president Nico...,[PARIS (Reuters) - Former French president Nic...,[paris (reuters) - former french president nic...,"[[-0.026092488294504824, 0.01578283199400815, ...","[[0.0, 0.9452452071803821, 0.9400790618462382,...","it would be misplaced.” nonetheless, each new..."
4,Paris Hilton arrived at LAX Wednesday dressed ...,[Paris Hilton arrived at LAX Wednesday dressed...,[paris hilton arrived at lax wednesday dressed...,"[[0.03635012516146811, -0.019238118671135677, ...","[[0.0, 0.9470859283118694, 0.8958751258387555,...",monty died sunday after a long battle with can...
5,"BERLIN, June 17 (Reuters) - ECB board member B...","[BERLIN, June 17 (Reuters) - ECB board member ...","[berlin, june 17 (reuters) - ecb board member ...","[[-0.017836371216385035, -0.048343966650413996...","[[0.0, 0.9645579331415346, 0.9602312190392982]...",
6,CARACAS (Reuters) - Venezuelan authorities hav...,[CARACAS (Reuters) - Venezuelan authorities ha...,[caracas (reuters) - venezuelan authorities ha...,"[[0.008267677637270963, -0.0019148314182220057...","[[0.0, 0.9180468711823176, 0.8926102809426495,...",they also come on the heels of a visit to vene...
7,If only every day could be like this. You can’...,"[If only every day could be like this., You ca...","[if only every day could be like this., you ca...","[[-0.0172803283239598, 0.042179181784842305, 0...","[[0.0, 0.9406811720981931, 0.9331990333560202,...",if it’s too difficult to take the emotion out ...
8,"Google I/O, the company's big developer confer...","[Google I/O, the company's big developer confe...","[google i/o, the company's big developer confe...","[[0.0012241876734280456, -0.004594854212485783...","[[0.0, 0.8383653692341118, 0.8196385596407044,...",and follow our coverage throughout the week fo...
9,China is dismissing unfavorable media reports ...,[China is dismissing unfavorable media reports...,[china is dismissing unfavorable media reports...,"[[-0.08900843953635339, -0.06765881264492032, ...","[[0.0, 0.9607754105320491, 0.9722333117894462,...","in his short political career, donald trump ha..."


In [ ]:
df['article'][1]

" The Indianapolis Colts made Andrew Luck the highest-paid player in NFL history this offseason with a five-year, $122-million contract with $89 million guaranteed. However, they're already finding that Luck's contract is inhibiting their ability to address weaknesses on other parts of the roster, particularly on defense. On Friday, Colts GM Ryan Grigson, who is under fire for the Colts 1-3 start, said that it's difficult to build up the team's defense with Luck making so much money. According to Keefer, Grigson did point out that the Colts still have young talent they're hoping to develop on defense. However, blaming Luck's contract — which the Colts gave him — for having a weak defense (30th in defensive DVOA) is not accurate. As others have pointed out, last year's Denver Broncos paid Peyton Manning $15 million in base salary while also boasting an elite defense. Luck also takes up $18.4 million against the salary cap this year, less than $1 million more than Manning did a year ago 

In [ ]:
df['summary'][1]

"much of this comes from drafting successfully, which the colts have not done as well as elite teams like the broncos or the patriots, for instance. on friday, colts gm ryan grigson, who is under fire for the colts 1-3 start, said that it's difficult to build up the team's defense with luck making so much money."